# LaZerRanks Glicko

In [1]:
using JuliaDB, IterableTables, Dates, Printf;

In [2]:
function makeCompDict(wcafile)
    compids = String[]
    compdates = Date[]
    lines = readlines(wcafile)
    for i = 2:length(lines) # skip header line
        line = lines[i]
        data = split(line, "\t")
        push!(compids, data[1])
        compdate = string(data[6], "-", data[7], "-", data[8])
        push!(compdates, Date(compdate, "y-m-d"))
    end
        compDict = Dict(zip(compids, compdates))
        sortedCompsArray = sort(collect(zip(values(compDict),keys(compDict))))
    
    return compDict, sortedCompsArray
end


makeCompDict (generic function with 1 method)

In [3]:
results = loadndsparse("WCA_export_Results.tsv"; delim = '\t',
          indexcols = [:competitionId, :eventId, :roundTypeId, :personId],
          colparsers=Dict("personId"=>String,"eventId"=>String,"competitionId"=>String, "roundTypeId"=>String, "pos" =>Int))

4-d NDSparse with 1965612 values (13 field named tuples):
    Dimensions#  colname        type
────────────────────────
1  competitionId  String
2  eventId        String
3  roundTypeId    String
4  personId       String
    Values#   colname                type
─────────────────────────────────
5   pos                    Int64
6   best                   Int64
7   average                Int64
8   personName             String
9   personCountryId        String
10  formatId               String
11  value1                 Int64
12  value2                 Int64
13  value3                 Int64
14  value4                 Int64
15  value5                 Int64
16  regionalSingleRecord   String
17  regionalAverageRecord  String

In [1]:
function genGlickoArray(comp, event, round)
    
    myresults = table(results[comp, event, round, :])
    mytable = table(select(myresults, (:personId, :pos)), pkey=:pos)
    
    return mytable
end
    

genGlickoArray (generic function with 1 method)

In [5]:
function genAllGlickoArrays(event)
    compDict, sortedCompsArray = makeCompDict("WCA_export_Competitions.tsv")
    allGlickoArrays = []
    lengthOfComps = length(sortedCompsArray)
    for i in 1:length(sortedCompsArray)
        index = sortedCompsArray[i][1]
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "0"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "1"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "2"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "3"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "b"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "c"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "d"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "e"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "f"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "g"))
        push!(allGlickoArrays, genGlickoArray(sortedCompsArray[i][2], event, "h"))
        print("\r $i / $lengthOfComps Comps  ||  $index")
    end
    output = []
    for i in 1:length(allGlickoArrays)
        if length(allGlickoArrays[i]) != 0
            push!(output, allGlickoArrays[i])
        end
    end
    return output
end

genAllGlickoArrays (generic function with 1 method)

In [6]:
function findSigma(mu, phi, sigma, delta, v)
    tau = 0.5
    epsilon = 0.00001
    
    A = a = log(sigma^2)
        
    f(x) = exp(x)*(delta^2 - phi^2 - v - exp(x))/(2*(phi^2 + v + exp(x))^2) - (x - a)/(tau^2)

    if delta^2 > phi^2 + v
        B = log(delta^2 - phi^2 - v)
    else
        k = 1
        while f(a - k*tau) < 0
            k += 1
        end
        B = a - k*tau 
    end
    fA = f(A)
    fB = f(B)
    count = 0
    #println("")
    #println("count=$count: A=$A B=$B, |B-A|=", abs(B-A), " ϵ=$epsilon")

    while abs(B-A) > epsilon
        count += 1
        #println("count=$count: A=$A B=$B, |B-A|=", abs(B-A), " ϵ=$epsilon")
        C = A + ((A - B)*fA)/(fB - fA)
        fC = f(C)
        if fC*fB < 0 
            A = B
            fA = fB
        else
            fA = fA/2
        end
        B = C
        fB = fC
        #println("count=$count: A=$A B=$B, |B-A|=", abs(B-A), " ϵ=$epsilon")
    end

    return exp(A/2)   
end

findSigma (generic function with 1 method)

In [7]:
function calcRatings(data, ratingdict, RDdict, voldict)
    tau = 0.5
    conv = 173.7178
    
    g(oppPhi) = 1/sqrt(1 + 3*oppPhi^2/pi^2)
    E(mu, oppMu, oppPhi) = 1/(1 + exp(-g(oppPhi)*(mu - oppMu)))
        
    updateRatings = []
    updateRDs = []
    updateVols = []

    for i in 1:length(data)

        mu = (ratingdict[data[i].personId] - 1500.0)/conv
        phi = RDdict[data[i].personId]/conv
        sigma = voldict[data[i].personId]
        v_inv = 0.0
        change = 0.0

        println("i=$i mu=$mu phi=$phi sigma=$sigma")
        for j in 1:length(data)
            if i != j
                oppMu = (ratingdict[data[j].personId] - 1500.0)/conv
                oppPhi = RDdict[data[j].personId]/conv

                if data[i].pos > data[j].pos
                    S = 0
                elseif data[i].pos < data[j].pos
                    S = 1
                else
                    S = 0.5
                end

                v_inv += g(oppPhi)^2*E(mu, oppMu, oppPhi)*(1-E(mu, oppMu, oppPhi))
                change += g(oppPhi)*(S - E(mu, oppMu, oppPhi))
            end
        end
        if v_inv != 0
            v = 1/v_inv
        else
            v = 0.0
        end
        
        delta = v*change
        newSigma = findSigma(mu, phi, sigma, delta, v)
        phiAst = sqrt(phi^2 + newSigma^2)
        newPhi = 1 / sqrt(1/phiAst^2 + 1/v)
        newMu = mu + (newPhi^2)*delta

        push!(updateRatings, newMu*conv + 1500.0)
        push!(updateRDs, newPhi*conv)
        push!(updateVols, newSigma)
    end

    for i in 1:length(data)
        ratingdict[data[i].personId] = updateRatings[i]
        RDdict[data[i].personId] = updateRDs[i]
        voldict[data[i].personId] = updateVols[i]
    end
end

calcRatings (generic function with 1 method)

In [9]:
path_persons = "WCA_export_Persons.tsv"
wca_persons = loadtable(path_persons; delim = '\t', quotechar = '\\',
    colparsers=Dict("id"=>String,"subid"=>Int, "name"=>String, "CountryId"=>String, "gender"=>String));

persons = loadndsparse(path_persons; delim = '\t', quotechar = '\\', indexcols = [:id, :subid],
    colparsers=Dict("id"=>String,"subid"=>Int, "name"=>String, "CountryId"=>String, "gender"=>String));

In [19]:
function generateRatings(thing) 
    playerids = []
    for i in 1:length(wca_persons)
        push!(playerids, wca_persons[i][1])
    end

    #reset ratings
    ratings = []
    RDs = []
    vols = []
    for i in 1:length(playerids)
        push!(ratings, 1500.0)
        push!(RDs, 35.0)
        push!(vols, 0.06)
    end

    ratingdict = Dict(zip(playerids, ratings))
    RDdict = Dict(zip(playerids, RDs))
    voldict = Dict(zip(playerids, vols));
    
    for i in 1:length(thing)
        calcRatings(thing[i], ratingdict, RDdict, voldict)
        print("\r$i / ", length(thing))
    end
    
    RatingList = sort(collect(zip(values(ratingdict),keys(ratingdict))), rev = true)
    
    #=
    htb = @sprintf "%4s %-30s \t %-7s %-3s %-15s" "Rank" "Name" "Rating" "RD" "Citizen of"
    println()
    println(htb)
    for i in 1:length(RatingList)
        id = RatingList[i][2]
        name = persons[id, 1].name
        if length(name) > 30
            name = name[1:30]*"..."
        end
        rating = RatingList[i][1]
        RD = RDdict[id]
        country = persons[id, 1].countryId
        tb = @sprintf "%4d %30s \t %6.2f %-3d %-15s" i  rpad(name, 30) rating RD rpad(country, 15)
        println(tb)
    end
    =#
    return RatingList
end

generateRatings (generic function with 1 method)

In [11]:
thing = genAllGlickoArrays("333")

 5569 / 5569 Comps  ||  2019-07-06

14931-element Array{Any,1}:
 Table with 19 rows, 2 columns:
personId      pos
─────────────────
"1982THAI01"  1
"1982RAZO01"  2
"1982LABA01"  3
"1982PETR01"  4
"1982UENO01"  5
"1982JEAN01"  6
"1982CHIL01"  7
"1982TRIN01"  8
"1982ROME01"  9
"1982FRID01"  10
"1982VALD01"  11
"1982LAET01"  12
"1982BORS01"  13
"1982BRIN01"  14
"1982SAND01"  15
"1982GALR01"  16
"1982SEBE01"  17
"1982TENE01"  18
"1982TRAJ01"  19                                                                                  
 Table with 83 rows, 2 columns:
personId      pos
─────────────────
"2003KNIG01"  1
"2003AKIM01"  2
"2003ALLE01"  3
"2003VAND01"  4
"1982FRID01"  5
"2003BOND01"  6
"2003MAKI01"  7
"2003LIDO01"  8
"2003CAMA01"  9
"2003KONI01"  10
"2003BRUC01"  11
"2003WESL01"  12
⋮
"2003GOOD01"  73
"2003BRAN03"  74
"2003KNIG02"  75
"2003ALGA01"  76
"2003THOM01"  77
"2003JOZW01"  78
"2003STAU01"  79
"2003TING01"  80
"2003ZBOR01"  81
"2003GRAN01"  82
"2003POUR01"  83            
 Table with 32 rows, 2 columns:
personId    

In [14]:
calcRatings(thing[874], ratingdict, RDdict, voldict)

i=1 mu=0.0 phi=2.014761872416068 sigma=0.06
i=2 mu=0.0 phi=2.014761872416068 sigma=0.06
i=3 mu=0.0 phi=2.014761872416068 sigma=0.06
i=4 mu=0.0 phi=2.014761872416068 sigma=0.06
i=5 mu=0.0 phi=2.014761872416068 sigma=0.06
i=6 mu=0.0 phi=2.014761872416068 sigma=0.06
i=7 mu=0.0 phi=2.014761872416068 sigma=0.06
i=8 mu=0.0 phi=2.014761872416068 sigma=0.06
i=9 mu=0.0 phi=2.014761872416068 sigma=0.06
i=10 mu=0.0 phi=2.014761872416068 sigma=0.06
i=11 mu=0.0 phi=2.014761872416068 sigma=0.06
i=12 mu=0.0 phi=2.014761872416068 sigma=0.06
i=13 mu=0.0 phi=2.014761872416068 sigma=0.06
i=14 mu=0.0 phi=2.014761872416068 sigma=0.06
i=15 mu=0.0 phi=2.014761872416068 sigma=0.06
i=16 mu=0.0 phi=2.014761872416068 sigma=0.06
i=17 mu=0.0 phi=2.014761872416068 sigma=0.06
i=18 mu=0.0 phi=2.014761872416068 sigma=0.06
i=19 mu=0.0 phi=2.014761872416068 sigma=0.06
i=20 mu=0.0 phi=2.014761872416068 sigma=0.06
i=21 mu=0.0 phi=2.014761872416068 sigma=0.06
i=22 mu=0.0 phi=2.014761872416068 sigma=0.06
i=23 mu=0.0 phi=2.0

In [13]:
playerids = []
    for i in 1:length(wca_persons)
        push!(playerids, wca_persons[i][1])
    end

    #reset ratings
    ratings = []
    RDs = []
    vols = []
    for i in 1:length(playerids)
        push!(ratings, 1500.0)
        push!(RDs, 350.0)
        push!(vols, 0.06)
    end

    ratingdict = Dict(zip(playerids, ratings))
    RDdict = Dict(zip(playerids, RDs))
    voldict = Dict(zip(playerids, vols));

In [15]:
sort(collect(zip(values(ratingdict),keys(ratingdict))), rev = true)

116207-element Array{Tuple{Any,Any},1}:
 (1562.5346828192305, "2009HAYS01")
 (1560.7976082764028, "2008BRID01")
 (1559.060533735272, "2007ALMA01") 
 (1557.3234591957894, "2006ADAM01")
 (1555.5863846579064, "2007CHOW01")
 (1553.8493101215747, "2008NGUY03")
 (1552.1122355867456, "2006MOND01")
 (1550.3751610533707, "2005ALBR01")
 (1548.638086521401, "2007SHID01") 
 (1546.9010119907884, "2008WARL01")
 (1545.1639374614842, "2007MENG01")
 (1543.4268629334401, "2007CHEN15")
 (1541.6897884126681, "2009HAYA01")
 ⋮                                 
 (1456.5731370665599, "2009KESI01")
 (1454.8360625385158, "2009ROUT01")
 (1453.0989880092116, "2009KANG02")
 (1451.361913478599, "2007HUGH04") 
 (1449.6248389466293, "2007HUGH03")
 (1447.8877644132544, "2007ADAM01")
 (1446.1506898784253, "2009AMBA01")
 (1444.4136153420936, "2009TRUO01")
 (1442.6765408042106, "2009HANS02")
 (1440.939466264728, "2009SING03") 
 (1439.2023917235972, "2009DIAM01")
 (1437.4653171807695, "2009TSEJ01")

In [20]:
generateRatings(thing[1:873])

i=1 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=2 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=3 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=4 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=5 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=6 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=7 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=8 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=9 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=10 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=11 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=12 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=13 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=14 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=15 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=16 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=17 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=18 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=19 mu=0.0 phi=0.20147618724160676 sigma=0.06
1 / 873i=1 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=2 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=3 mu=0.0 phi=0.

i=7 mu=0.014010984413153198 phi=0.18631384558413278 sigma=0.05998629264594404
i=8 mu=0.008426622141535656 phi=0.18630729183922481 sigma=0.059953295722690585
i=9 mu=0.04198186572070078 phi=0.18639732942698872 sigma=0.06040525257332728
i=10 mu=0.05319289383526091 phi=0.18645601548803659 sigma=0.06069826830740575
i=11 mu=0.025199203549700462 phi=0.18633693066370077 sigma=0.060102396867568984
i=12 mu=-0.019263375409911923 phi=0.1532376600406212 sigma=0.06345926552820645
7 / 873i=1 mu=0.13877932348947494 phi=0.16131886119052485 sigma=0.06763181541067052
i=2 mu=0.14265034810881674 phi=0.15590876981200927 sigma=0.06261479422088734
i=3 mu=0.055998825525942236 phi=0.15717489734049525 sigma=0.06233348479076292
i=4 mu=0.055445822290399883 phi=0.18638093942684728 sigma=0.060277958842872806
i=5 mu=0.07894925933654724 phi=0.1865023105035624 sigma=0.06057654090478575
8 / 873i=1 mu=0.1968702521963019 phi=0.1723501119933872 sigma=0.06768160241894014
i=2 mu=0.16859408724252417 phi=0.16571260919324768 si

i=6 mu=0.015527497990269845 phi=0.14404880670168643 sigma=0.06751289760762995
i=7 mu=0.30345166002070717 phi=0.15877049051914666 sigma=0.06504541320492482
i=8 mu=0.03677521045818824 phi=0.14442612776450336 sigma=0.06668219846434287
i=9 mu=0.03827277874453709 phi=0.14231239327223394 sigma=0.06293189159598721
i=10 mu=0.012483671480666406 phi=0.14363117554900137 sigma=0.06028101483161816
i=11 mu=0.012280647305240254 phi=0.14409819091659587 sigma=0.06200805000085439
i=12 mu=0.04689835854384351 phi=0.17181602127902784 sigma=0.061999592733969065
i=13 mu=0.05101146650295953 phi=0.15217326298476797 sigma=0.06206083290379273
i=14 mu=0.018403781753488817 phi=0.17153317445116106 sigma=0.06020189507115326
i=15 mu=0.06255237650468552 phi=0.1625254169137628 sigma=0.06173352193069783
i=16 mu=0.0055273831815340875 phi=0.1714902775277444 sigma=0.059925338603515135
i=17 mu=0.023562713065257725 phi=0.17156392078760355 sigma=0.060399465080190284
i=18 mu=0.07030904650784657 phi=0.15768920427994845 sigma=0.

i=6 mu=0.25929890306826026 phi=0.16771081889187048 sigma=0.06536834656515825
i=7 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=8 mu=-0.047441319229108374 phi=0.1573499588113054 sigma=0.06260190305576141
i=9 mu=-0.04917721628566004 phi=0.1596998727633184 sigma=0.06091138562411939
i=10 mu=0.05729368941834871 phi=0.16673554486752268 sigma=0.06231894806780812
i=11 mu=-0.028992413252264974 phi=0.16165976639111015 sigma=0.06005733920639634
i=12 mu=-0.018074129017642757 phi=0.1595219655056478 sigma=0.060329340626152986
i=13 mu=-0.013660502098672889 phi=0.16553642033587548 sigma=0.060704232039784395
i=14 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=15 mu=-0.026734320210292605 phi=0.15774146391303956 sigma=0.06083057599233354
i=16 mu=-0.06904277392124451 phi=0.16218834257874834 sigma=0.06099452565775572
i=17 mu=-0.0047599917308744815 phi=0.17148919091585577 sigma=0.059918319350315455
i=18 mu=0.021299458946339975 phi=0.15841872543428906 sigma=0.06177262972890774
i=19 mu=0.0 phi=0.20147618724160676 

i=17 mu=0.012863659864150393 phi=0.15476629367692296 sigma=0.06174954979578612
i=18 mu=-0.08363028244712228 phi=0.15755323257780332 sigma=0.062411589443305056
i=19 mu=-0.018496805222696547 phi=0.16529981026868257 sigma=0.061502749865306246
i=20 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=21 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=22 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=23 mu=-0.025232528690408765 phi=0.1529097051064886 sigma=0.061767162591957664
i=24 mu=-0.04089390964170199 phi=0.17173142378622622 sigma=0.061466494022290784
i=25 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=26 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=27 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=28 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=29 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=30 mu=-0.06193800682689302 phi=0.17213381882379725 sigma=0.0630172137441139
i=31 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=32 mu=-0.03754184407062245 phi=0.19399868424880043 sigma=0.06010692807957752
i=33 mu=0.0 phi=0.20

...

i=20 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=21 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=22 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=23 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=24 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=25 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=26 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=27 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=28 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=29 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=30 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=31 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=32 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=33 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=34 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=35 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=36 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=37 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=38 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=39 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=40 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=41 mu=0.0 p

i=4 mu=0.03855589595457468 phi=0.17446036926059694 sigma=0.06191519175740678
i=5 mu=0.09645161230863121 phi=0.18380082340141662 sigma=0.06089294136371399
i=6 mu=-0.028721610062895402 phi=0.17907691270573692 sigma=0.06017728678390778
i=7 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=8 mu=0.011337764528110469 phi=0.17910927613571223 sigma=0.060182706784436275
i=9 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=10 mu=0.03358982375860011 phi=0.17906901747041457 sigma=0.06006428928965164
i=11 mu=-0.020041673822037667 phi=0.17902961624836528 sigma=0.059988191886386405
i=12 mu=-0.031757782754317676 phi=0.17904796306959211 sigma=0.06006008706217811
i=13 mu=-0.09054024203556514 phi=0.17931017087642065 sigma=0.060963788289798655
i=14 mu=0.016954884217408067 phi=0.17902148857822417 sigma=0.059931799073637146
i=15 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=16 mu=-0.07380053222422264 phi=0.1792080133144478 sigma=0.06064576013165518
i=17 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=18 mu=0.0 phi=0.2014761

Excessive output truncated after 524299 bytes.

i=40 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=41 mu=-0.03155916855814446 phi=0.14206316089464827 sigma=0.061930016884579
i=42 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=43 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=44 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=45 mu=0.014046295578809948 phi=0.14208331521982495 sigma=0.06270060374559885
i=46 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=47 mu=-0.01572786954272072 phi=0.16180422193019484 sigma=0.0602936936420971
i=48 mu=-0.011536609801618504 phi=0.14547722914426697 sigma=0.05986478053280902
i=49 mu=0.0007738436021950308 phi=0.1328441167925763 sigma=0.06015993429541508
i=50 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=51 mu=-0.0191086909469736 phi=0.15981518200922196 sigma=0.060574282530357966
i=52 mu=-0.00603839656100747 phi=0.15344538938132438 sigma=0.06293959914289121
i=53 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=54 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=55 mu=0.0 phi=0.20147618724160676 sigma=0.06
i=56 mu=0.0 phi=0.20147618

116207-element Array{Tuple{Any,Any},1}:
 (6.604405670785742e57, "2005AKKE01")  
 (92599.0481135771, "2007NAKA03")      
 (2643.861360070169, "2004CHAM01")     
 (2606.7479112138667, "2007POLK01")    
 (2066.342908800122, "2004PONS01")     
 (2023.1604570828054, "2007ESPI01")    
 (2023.140647934295, "2005BATI01")     
 (2001.3498441992215, "2007LIME01")    
 (1988.2406289307705, "2006TEMM01")    
 (1986.3009349738745, "2007SUNT01")    
 (1981.268759439669, "2004LOLE01")     
 (1981.0399765272996, "2007HESS01")    
 (1934.4568604299761, "2006TSAO01")    
 ⋮                                     
 (1429.694807584119, "2003LARS01")     
 (1425.591220979848, "2006NORS01")     
 (1421.993885487118, "2006REYN01")     
 (1419.6270255003876, "2006SAWL01")    
 (1418.9756379115909, "2008RIJK01")    
 (1413.2793555428852, "2006MATH01")    
 (1408.2839874511003, "2008COUR01")    
 (1398.354722960227, "2007OEYM01")     
 (1354.2162167718961, "2008ZHUA01")    
 (1348.9476294611459, "2006SZWA01")    


In [ ]:
sum = 0
for i in 1:length(ratings)
    sum += ratings[i][1]
end
println("mean: ", sum/length(ratings))
println("median: ", ratings[Int.(length(ratings)/2)][1])
println("min: ", ratings[length(ratings)][1])
println("max: ", ratings[1][1])